This Jupyter Notebook contains the first project for Information Retrieval 1 taught at the UvA. Code is made by Oscar Ligthart, Nicole Ferreira Silverio and Arend van Dormalen.

ANSWER TO THEORETICAL QUESTION 1A

The chance of a type 1 error ($\alpha$) increases each time an experiment is repeated, if it's not corrected. The new $\alpha$ for _m_ experiments is $1 − (1 − \alpha)^m ≈ m\alpha$.

ANSWER TO THEORETICAL QUESTION 2

Assume two ranked lists created by two different rankers. List $l1$ contains documents $d1$, $d2$ and $d3$ in that order. List $l2$ contains documents $d2$, $d3$ and $d4$ in that order. Now assume that the only relevant document is $d3$, which will therefore be clicked on most often. From our judgment, it is obvious that $l2$ is the most relevant list as it has placed $d3$ on a higher position. However, in Team Draft Interleaving, these algorithms will be evaluated as having equal performance.

In this situation, $d3$ will always be the third item on the interleaved list. After the first coin flip, $d2$ will be removed from $l1$ as this document has already been supplied by $l2$.  At the second coin flip, $d3$ will be the next document for both lists. This causes the relevance for both lists to be the same, as they now both have the same chance of supplying the only relevant document to the interleaved list.

In [22]:
import itertools
import numpy as np
import random
import re

# first get the sequence options
relevance = ['N', 'R', 'HR']
options = list(itertools.product(relevance, repeat = 5))

# create all possible pairs in sequence options
pair_index = list(itertools.permutations(range(len(options)), 2))


pairs = []

for temp_pair in pair_index:
    pairs.append([options[temp_pair[0]], options[temp_pair[1]]])    


In [24]:
def get_average_precision(ranking):
    rel = 0
    AP_numerator = 0
    # get amount of relevant documents
    for i, doc in enumerate(ranking):
        if doc == 'R' or doc == 'HR':
            rel += 1
            AP_numerator += rel/(i+1)
            
    return rel, AP_numerator

# new dict for average precision for both P and E (key is pair, value is average precisions))
AP_delta = {}

# new dict for delta measures (AP, nDCG and ERR will be stored here per pair)
delta_values = {}

# get precision for all pairs
for pair in pairs:
    # first calculate numerator for average precision for P
    P = pair[0]    
    P_rel, P_AP_numerator = get_average_precision(P)
 
    # now calculate numerator for average precision for E
    E = pair[1]
    E_rel, E_AP_numerator = get_average_precision(E)

    # get total number of relevant documents returned from query
    total_rel = P_rel + E_rel
    
    # calculate average precision for both P and E
    P_AP = P_AP_numerator/total_rel
    E_AP = E_AP_numerator/total_rel
    
    # store results in a dict
    AP_delta[(P,E)] = E_AP - P_AP
    
    # store AP delta measures in list format in dict
    delta_values[(P,E)] = [E_AP - P_AP]

#for value in AP_results.values():
#    print(value)
    
#print(len(AP_results.values()))
print(len(pairs))

# ####### NOW GET DELTA MEASURES #########
# AP_delta_values = []
# for key, value in AP_results.items():
#     if value[1] > value[0]:
#         delta_value = value[1] - value[0]
#         AP_delta_values.append(delta_value)

# print(len(AP_delta_values))


58806


In [25]:
##### nDCG cell #####
def get_nDCG(ranking):
    DCG = 0
    
    # loop through ranking
    for i, rank in enumerate(ranking):
        # decide what the relative rank is
        if rank == 'HR':
            rel_r = 2
        elif rank == 'R':
            rel_r = 1
        elif rank == 'N':
            rel_r = 0
        
        DCG += (2**rel_r - 1)/(np.log2(1+(i+1)))
    
    return DCG

# new dict for average precision for both P and E (key is pair, value is average precisions))
nDCG_delta = {}

counter = 0
same_counter = 0
lower_counter = 0

# get nDCG for all pairs
for pair in pairs:    
    # first for P
    P = pair[0]
    P_DCG = get_nDCG(P)
    
    # then for E
    E = pair[1]
    E_DCG = get_nDCG(E)
    
    nDCG_delta[(P,E)] = E_DCG - P_DCG
    
    # add nDCG delta measure to dict
    delta_values[(P,E)].append(E_DCG - P_DCG)



In [26]:
##### ERR cell #####

def get_ERR(ranking):
    
    ERR = 0
    p = 1
    max_rel = 2
    
    # loop through ranking
    for i, rank in enumerate(ranking):
        
        # start at second rank
        if i != 0:
            
            # decide what the relative rank is
            if rank == 'HR':
                rel_r = 2
            elif rank == 'R':
                rel_r = 1
            elif rank == 'N':
                rel_r = 0

            # Calculate R with the mapping function
            R = (2**rel_r - 1)/(2**max_rel)

            # Modify ERR value
            ERR += p * (R/i)

            # Modify p
            p = p*(1-R)
    
    return ERR

# new dict for ERR values for both P and E (key is pair, value is ERR value))
ERR_delta = {}

# get ERR for all pairs
for pair in pairs:
    
    # first for P
    P = pair[0]
    P_ERR = get_ERR(P)
    
    # then for E
    E = pair[1]
    E_ERR = get_ERR(E)
    
    ERR_delta[(P,E)] = E_ERR - P_ERR
    
    # add ERR delta measures to dict
    delta_values[(P,E)].append(E_ERR -  P_ERR)
    
# ##### NOW GET THE DELTA MEASURES #####
# ERR_delta_values = []
# for key, value in ERR_results.items():
#     if value[1] > value[0]:
#         delta_value = value[1] - value[0]
#         ERR_delta_values.append(delta_value)

# print(len(ERR_delta_values))


In [27]:
##### Filter pairs #####

def filter_all_pairs(pairs_dict):
    
    pairs_list = []
    dict_items = pairs_dict.items()
    for pair in dict_items:
        scores = pair[1]
        avg = sum(scores, 0.0)/len(scores)
        if avg > 0.0:
            pairs_list.append(pair[0])
    
    return pairs_list

def filter_metric_pairs(pairs_dict):
    
    pairs_list = []
    dict_items = pairs_dict.items()
    for pair in dict_items:
        score = pair[1]
        if score > 0.0:
            pairs_list.append(pair[0])
    
    return pairs_list


# Iterate through pairs
def interleaving(pairs):
    for pair in pairs:

        # Flip a coin, assign winning and losing
        # P = pair[0], E = pair[1]
        coin_winner = random.randint(0,1)
        winner = pair[coin_winner]
        loser = pair[1 - coin_winner]

        # initiate lists
        resulting_list = []
        origin_list = []

        # iterate through lists, fill up results and origin list
        for i in range(len(winner)):
            resulting_list.append(winner[i])
            origin_list.append(coin_winner)
            resulting_list.append(loser[i])
            origin_list.append(1-coin_winner)

        all_results.append(resulting_list)
        all_origins.append(origin_list)
    
    return all_results, all_origins

In [30]:
##### Balanced Interleaving #####

all_results = []
all_origins = []

print("before:", len(delta_values.items()))

all_pairs = filter_all_pairs(delta_values)

print("after:", len(all_pairs))

all_results, all_origins = interleaving(all_pairs)

# now get pairs for which delta measure is positive for every offline metric
AP_pairs = filter_metric_pairs(AP_delta)
DCG_pairs = filter_metric_pairs(nDCG_delta)
ERR_pairs = filter_metric_pairs(ERR_delta)

# interleave the pairs for which E outperforms P per offline metric
AP_results, AP_origins = interleaving(AP_pairs)
DCG_results, DCG_origins = interleaving(DCG_pairs)
ERR_results, ERR_origins = interleaving(ERR_pairs)



before: 58806
after: 29403


In [19]:
##### Random Click Model #####

# Learns parameter
def learn_param_RCM(data):
    
    # open file and read
    lines=data.readlines()

    clicks = 0
    documents = 0

    # Acquire total amount of queries and clicks
    for line in lines:
        items = re.split(r'\t+',line)
        if items[2] == "Q":
            # Per query 10 documents are shown
            documents += 10
        elif items[2] == "C":
            clicks += 1
    
    # Calculate rho
    rho = clicks/documents
    
    return rho

# Predicts a click probability
def predict_prob_RCM(ranking, param):
    
    # get the click probability for every document in ranking
    click_prob = []
    for doc in ranking:
        click_prob.append(param)
        
    return click_prob

# Decide whether document is clicked on
def click_doc_RCM(click_prob):
    clicked = []
    for prob in click_prob:
        chance = random.random()
        if chance <= prob:
            clicked.append(1)
        else:
            clicked.append(0)
    return clicked

def RCM_simulation(pairs, origins, rho):

    N = 50
    p_RCM_list = []

    # simulate experiment N times
    for i in range(N):
        # keep track of which algorithm won
        E_win = 0
        P_win = 0
        # loop through all rankings
        for j, ranking in enumerate(pairs):

            # predict probability of clicking
            click_prob = predict_prob_RCM(ranking, rho)

            # get which documents were clicked
            clicked = click_doc_RCM(click_prob)

            # now shuffle the origin list so documents are picked at random
            origin_shuffle = random.sample(origins[j], len(origins[j]))

            E_click = 0
            P_click = 0
            # check whether the clicked documents were produced by E or P
            for h, click in enumerate(clicked):
                if click == 1 and origin_shuffle[h] == 1:
                    E_click += 1
                elif click == 1 and origin_shuffle[h] == 0:
                    P_click += 1

            # determine whether E or P won
            if E_click > P_click:
                E_win += 1
            elif P_click > E_click:
                P_win += 1

        # proportion of times E won
        p = E_win / (E_win + P_win)
        p_RCM_list.append(p)
        
    return p_RCM_list


In [21]:
###### Simulate random click model ######

# get parameter out of data
f=open("YandexRelPredChallenge.txt","r")
rho = learn_param_RCM(f)
f.close()

# get the p for the average of all metrics
p_RCM_list = RCM_simulation(all_results, all_origins, rho)

# get the p for every metric 
p_AP_RCM_list = RCM_simulation(AP_results, AP_origins, rho)
p_DCG_RCM_list = RCM_simulation(DCG_results, DCG_origins, rho)
p_ERR_RCM_list = RCM_simulation(ERR_results, ERR_origins, rho)

print(p_RCM_list)
print(p_AP_RCM_list)
print(p_DCG_RCM_list)
print(p_ERR_RCM_list)


[0.49436952592798555, 0.49521531100478466, 0.4997851926106258, 0.49776753564741466, 0.5053093586294776, 0.5152465089769165, 0.5067662813645334, 0.5028514399771885, 0.4995787700084246, 0.5034599632820224, 0.4995128740431454, 0.4893882646691635, 0.5059548830040633, 0.5009962994591517, 0.5016365447559413, 0.4992917847025496, 0.4969714044231582, 0.5032158836689038, 0.49437722419928826, 0.4915254237288136, 0.49584565554147303, 0.5095110610116951, 0.49971759390002823, 0.4970347359502965, 0.5003470776065528, 0.49929617117117114, 0.5111861544955677, 0.5029585798816568, 0.5081187010078387, 0.4923508771929825, 0.501287922152261, 0.5052264808362369, 0.4978601997146933, 0.5039425513939735, 0.5000696864111498, 0.5067328136073707, 0.4983832419513567, 0.5027569630991093, 0.4985091580292489, 0.5011454753722795, 0.487873660462493, 0.5018005540166205, 0.5037722419928825, 0.49755073477956613, 0.4990940766550523, 0.49235731313981207, 0.4955555555555556, 0.5104835258878905, 0.5045753503002574, 0.5002121940

In [32]:
##### Simple Dynamic Bayesian Model #####

# Learns parameter
def learn_param_DBM(file):
    
    lines = file.readlines()

    #previous_session = 0 # Keep track of session number to determine if click is last click.
    previous_type = ""
    
    clicks = 0
    #last_clicks_session = 0
    last_clicks_query = 0

    lines.reverse() # Reversed order, so it is detectable if a click is last.
    for line in lines:
        items = re.split(r'\t+',line) #strip tabs
        #current_session = items[0]
        current_type = items[2]
        #if current_type == "C" and current_session != previous_session:
            #last_clicks_session += 1
        if current_type == "C" and previous_type == "Q": 
            last_clicks_query += 1
        if current_type == "C":
            clicks += 1
        #previous_session = current_session
        previous_type = current_type

    sigma = last_clicks_query/clicks
    
    return sigma
        

# Predicts a click probability
def predict_prob_DBM(rank, sigma):
    # for the click probability, we'll need P(A) and P(E)

    # first get alpha, which will be set according to the level of relevance of a document
    if rank == 'HR':
        alpha = 0.9
    elif rank == 'R':
        alpha = 0.3
    elif rank == 'N':
        alpha = 0 
    
    # check if user will click on the document (depending on alpha)
    x = random.random()
    if x <= alpha:
        P_A = 1
    else:
        P_A = 0
            
    # since we are using a simple DBM, gamma will always be one    
    gamma = 1
    
    return P_A, gamma  
       
        
# Decide which documents are clicked
def click_doc_DBM(ranking, sigma):
    # this function takes a ranking list and a value for the parameter sigma as input and uses
    # these to determine which documents in the ranking list are clicked on
    
    # set P(E) to 1 (first snippet is always read)
    P_E = 1
    
    clicked = []
    
    # run through the ranking to decide whether a document will be clicked or not
    for rank in ranking:
        P_A, gamma = predict_prob_DBM(rank, sigma)
        
        # based on probability, set click to 1 or 0
        if P_A == 1 and P_E == 1:
            P_C = 1
        else:
            P_C = 0
     
        clicked.append(P_C)
        
        # if user has clicked, check if user is satisfied
        if P_C == 1:
            # now check if user is satisfied
            x = random.random()
            if x <= sigma:
                # if satisfied, user will not read any more snippets (thus click nothing)
                P_E = 0
            else:
                # if user is not satisfied, user will read next snippet (thus possibly click)
                P_E = 1 
        
    return clicked       

def DBM_simulation(pairs, origins, sigma):
    N = 50
    p_DBM_list = []
    # simulate experiment N times
    for i in range(N):

        # keep track of which algorithm won
        E_win = 0
        P_win = 0
        # loop through all rankings
        for j, ranking in enumerate(pairs):

            # get which documents were clicked
            clicked = click_doc_DBM(ranking, sigma)

            E_click = 0
            P_click = 0

            current_origin = origins[j]

            # check whether the clicked documents were produced by E or P
            for h, click in enumerate(clicked):
                if click == 1 and current_origin[h] == 1:
                    E_click += 1
                elif click == 1 and current_origin[h] == 0:
                    P_click += 1

            # determine whether E or P won
            if E_click > P_click:
                E_win += 1
            elif P_click > E_click:
                P_win += 1

        # proportion of times E won
        p = E_win / (E_win + P_win)
        p_DBM_list.append(p)
    return p_DBM_list

In [33]:
##### Simulate dynamic bayesian model #####

# get parameter out of data
f=open("YandexRelPredChallenge.txt","r")
sigma = learn_param_DBM(f)
f.close()

# get the p for the average of all metrics
p_DBM_list = DBM_simulation(all_results, all_origins, sigma)

# get the p for every metric 
p_AP_DBM_list = DBM_simulation(AP_results, AP_origins, sigma)
p_DCG_DBM_list = DBM_simulation(DCG_results, DCG_origins, sigma)
p_ERR_DBM_list = DBM_simulation(ERR_results, ERR_origins, sigma)

print(p_DBM_list)
print(p_AP_DBM_list)
print(p_DCG_DBM_list)
print(p_ERR_DBM_list)


[0.7099801518812565, 0.7129052806598936, 0.7123160674823479, 0.7114819615467703, 0.712980738120403, 0.7142148991885513, 0.7141347822805826, 0.7113804902796818, 0.7128573121720486, 0.7135255323971634, 0.7118952699062601, 0.7145257353337139, 0.7140700971355262, 0.7121856658230574, 0.7128362003037516, 0.7118042826644815, 0.7127129609661419, 0.7136626498239266, 0.7112596345604485, 0.7133348411903454, 0.7130272779159802, 0.7130649900613603, 0.7122066993904741, 0.7143057458122553, 0.7142949484502763, 0.7116025675602783, 0.7131125398454806, 0.711474312609977, 0.7143287986988507, 0.7115449016409381, 0.7114860480207658, 0.7118930861669451, 0.7134041084066978, 0.7145788115369248, 0.7143288670248344, 0.712850605652759, 0.7133319671689119, 0.7130246394619414, 0.7128311329709207, 0.7117740386172061, 0.71347381589164, 0.7121149985970516, 0.7123308351755001, 0.7120089835230095, 0.7133226740417441, 0.7133556467285683, 0.7139667335447544, 0.713138607458504, 0.7112032657292814, 0.7139397870385384]
[0.71